In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, CuDNNLSTM, Dropout

# change tensorflow default behavior (where it uses all of the memory at the outset)
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import numpy as np
import matplotlib.pyplot as plt
import random
from datetime import datetime as dt

# interactive graphs on jupyter notebook
import mpld3

# 1. Data Preparation
### 1.1 Populate Raw Data

In [ ]:
size_dim1 = 100
size_dim2 = 6
multiplication = 1000

raw_data = []
for i in range(size_dim1):
    tmp_list = [(i + j) * multiplication for j in range(size_dim2)]
    raw_data.append(np.reshape(tmp_list, (-1, 1)))

raw_data = np.array(raw_data, dtype=float)
raw_data_shape = raw_data.shape

### 1.2 Data Normalization

In [ ]:
scaler = MinMaxScaler()
data = scaler.fit_transform(raw_data.reshape(-1, 1))
data = data.reshape(raw_data_shape)

### 1.3 Input & Target Preparation

In [ ]:
inputs = [x[:size_dim2 - 1] for x in data]
targets = [x[size_dim2 - 1][0] for x in data]

inputs = np.array(inputs, dtype=float)
targets = np.array(targets, dtype=float)

### 1.4 Train & Test Preparation

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.2, random_state=4)

print('x_train.shape:\t', x_train.shape)
print('y_train.shape:\t', y_train.shape)

# 2. Neural Network
### 2.1 Model Definition

In [ ]:
model = Sequential()
# model.add(CuDNNLSTM(64, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
model.add(LSTM(units=64, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))

# model.add(CuDNNLSTM(64, return_sequences=True))
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(0.2))

# model.add(CuDNNLSTM(64))
model.add(LSTM(units=64))
model.add(Dropout(0.2))

model.add(Dense(units=1))

model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])
model.summary()

### 2.2 Train Model

In [ ]:
start_time = dt.now().strftime("%Y-%m-%d %H:%M:%S.%f")
history = model.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test))
end_time = dt.now().strftime("%Y-%m-%d %H:%M:%S.%f")

### 2.3 Result Prediction

In [ ]:
print(f'Train Start:\t{start_time}')
print(f'Train End:\t{end_time}')

results = model.predict(x_test)

# 3. Result Visualization
### 3.1 Loss Plotting

In [ ]:
plt.plot(history.history['loss'], c='g', label='loss')
plt.plot(history.history['val_loss'], c='b', label='val_loss')
plt.legend()
plt.show()

### 3.2 Result Plotting

In [ ]:
print('results.shape', results.shape)
print('y_test.shape', y_test.shape)

mpld3.enable_notebook()
plt.rcParams['figure.figsize'] = [6, 4]

plt.plot(range(results.shape[0]), results, c='r', marker='x', ls='none', label='Predict')
plt.plot(range(results.shape[0]), y_test, c='g', marker='+', ls='none', label='Real')
plt.legend()
plt.show()

### 3.3 Inverse Normalization

In [ ]:
scaler.inverse_transform(results.reshape(-1, 1))

# 4. Additional Test - Input Shuffle

In [ ]:
for x in x_test:
    random.shuffle(x.reshape(-1))
results = model.predict(x_test)

In [ ]:
plt.plot(range(results.shape[0]), results, c='r', marker='x', ls='none', label='Predict')
plt.plot(range(results.shape[0]), y_test, c='g', marker='+', ls='none', label='Real')
plt.legend()
plt.show()